In [3]:
from langchain.sql_database import SQLDatabase

db = SQLDatabase.from_uri("mysql+pymysql://exxxxp:@local/order_bot")

In [5]:
print(db.get_usable_table_names())

['orders']


In [7]:
print(db.get_table_info())


CREATE TABLE orders (
	order_id BIGINT(20) UNSIGNED NOT NULL AUTO_INCREMENT, 
	customer_name VARCHAR(100), 
	status VARCHAR(50), 
	delivery_date DATE, 
	refund_status VARCHAR(50), 
	payment_method VARCHAR(50), 
	PRIMARY KEY (order_id)
)COLLATE utf8mb4_general_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from orders table:
order_id	customer_name	status	delivery_date	refund_status	payment_method
1	Arshad Anwar	Delivered	2025-07-01	No Refund	Credit Card
2	Bob Smith	Pending	None	No Refund	PayPal
3	Charlie Brown	Shipped	2025-07-28	No Refund	Debit Card
*/


In [9]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile",
               api_key="",
               temperature=0,
               max_retries=2)

In [11]:
from langchain_core.prompts import ChatPromptTemplate

system_message = """
Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

Only use the following tables:
{table_info}
"""

user_prompt = "Question: {input}"

query_prompt_template = ChatPromptTemplate(
    [("system", system_message), ("user", user_prompt)]
)

In [13]:
for message in query_prompt_template.messages:
    message.pretty_print()

================================ System Message ================================


Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

Only use the following tables:
{table_info}

================================ Human Message =================================

Question: {input}


In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

# Create SQL Database Chain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_direct=True)
result = db_chain.invoke({"query": "Arshad Anwar order details?"})
print(result)

In [ ]:
from langchain.sql_database import SQLDatabase

db = SQLDatabase.from_uri("mysql+pymysql:///order_bot")


from langchain_groq import ChatGroq

# Initialize LLM (Ollama)
llm = ChatGroq(model="llama-3.3-70b-versatile",
               api_key="hided due to security",
               temperature=0,
               max_retries=2)


from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

# Create SQL Database Chain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_direct=True)
result = db_chain.invoke({"query": "Arshad Anwar order details?"})
print(result)

In [ ]:
import os
from dotenv import load_dotenv
from langchain.sql_database import SQLDatabase
from langchain_groq import ChatGroq
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
import logging

# ✅ Setup logging
logging.basicConfig(level=logging.INFO)

# ✅ Load environment variables
load_dotenv()

DB_URI = os.getenv("DB_URI", "_bot")
GROQ_API_KEY = ""

def main():
    try:
        # ✅ Connect to database
        db = SQLDatabase.from_uri(DB_URI)
        logging.info("✅ Database connected successfully")

        # ✅ Initialize LLM
        llm = ChatGroq(
            model="llama-3.3-70b-versatile",
            api_key=GROQ_API_KEY,
            temperature=0,
            max_retries=2
        )

        # ✅ Define a structured prompt template
        template = """
        You are an AI assistant that converts natural language questions into SQL queries
        and retrieves results from the database.
        
        Use the database schema and context to generate accurate SQL queries.
        Question: {question}
        """
        prompt = PromptTemplate(input_variables=["question"], template=template)

        # ✅ Create SQL Database Chain
        db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=False, return_direct=True)

        # ✅ Get user query
        user_query = "Arshad Anwar order details?"
        formatted_query = prompt.format(question=user_query)

        raw_result = result.get("result", "")
        clean_result = raw_result.strip("[]")
        print("\n✅ Clean Result:\n", clean_result)

    
    except Exception as e:
        logging.error(f"❌ Error: {e}")

if __name__ == "__main__":
    main()

In [ ]:
import os
from dotenv import load_dotenv
from langchain.sql_database import SQLDatabase
from langchain_groq import ChatGroq
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
import logging

# ✅ Setup logging
logging.basicConfig(level=logging.INFO)

# ✅ Load environment variables
load_dotenv()
DB_URI = os.getenv("DB_URI", "mysql+pymysql:/")
GROQ_API_KEY = ""

def main():
    try:
        # ✅ Connect to database
        db = SQLDatabase.from_uri(DB_URI)
        logging.info("✅ Database connected successfully")

        # ✅ Initialize LLM
        llm = ChatGroq(
            model="llama-3.3-70b-versatile",
            api_key=GROQ_API_KEY,
            temperature=0,
            max_retries=2
        )

        # ✅ Define structured prompt
        template = """
        You are an AI assistant that converts natural language questions into SQL queries
        and retrieves results from the database.
        
        Question: {question}
        """
        prompt = PromptTemplate(input_variables=["question"], template=template)

        # ✅ Create SQL Database Chain
        db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=False, return_direct=True)

        # ✅ User query
        user_query = "Arshad Anwar order details?"
        formatted_query = prompt.format(question=user_query)

        # ✅ Execute
        result = db_chain.invoke({"query": formatted_query})

        # ✅ Extract and clean result
        raw_result = result.get("result", "")
        clean_result = raw_result.strip("[]")
        print("\nResult:\n", clean_result)

    except Exception as e:
        logging.error(f"❌ Error: {e}")

if __name__ == "__main__":
    main()


In [3]:
!streamlit run app2.py

^C
